# Iceberg Table Views

## Start Polaris Spark Session

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import json
import pandas as pd
import boto3
import snowflake.connector
from pyspark.sql import SparkSession, DataFrame

In [ ]:
polaris_uri=os.getenv("POLARIS_URI")
polaris_credential=os.getenv("POLARIS_CREDENTIAL")
aws_key=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key=os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region=os.getenv("AWS_REGION")

In [ ]:
os.environ['PACKAGES'] = "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.awssdk:bundle:2.20.160,software.amazon.awssdk:url-connection-client:2.20.160"


In [ ]:
def polaris_spark_session():
    """Initialize SparkSession with Iceberg support."""
    from pyspark.sql import SparkSession, DataFrame
    spark = SparkSession.builder.appName('migration')\
    .config('spark.jars.packages', os.environ['PACKAGES'])\
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config('spark.sql.defaultCatalog','polaris')\
    .config('spark.sql.catalog.polaris','org.apache.iceberg.spark.SparkCatalog')\
    .config('spark.sql.catalog.polaris.type','rest')\
    .config('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation','true')\
    .config('spark.sql.catalog.polaris.uri',polaris_uri)\
    .config('spark.sql.catalog.polaris.credential',polaris_credential)\
    .config('spark.sql.catalog.polaris.warehouse','ambari_prod')\
    .config('spark.sql.catalog.polaris.scope','PRINCIPAL_ROLE:ALL')\
    .config('spark.sql.catalog.polaris.client.region',aws_region)\
    .config('spark.hadoop.fs.s3a.access.key',aws_key )\
    .config('spark.hadoop.fs.s3a.secret.key',aws_secret_key )\
    .config('spark.hadoop.fs.s3.impl','org.apache.hadoop.fs.s3a.S3AFileSystem' )\
    .getOrCreate()
    return spark 

In [ ]:
spark_polaris=polaris_spark_session()

In [ ]:
spark_polaris.sql("""use polaris.sales """).show(100,100)

In [ ]:
spark_polaris.sql("""SHOW Tables  """).show(100,100)

In [ ]:
spark_polaris.sql("""Desc  extended polaris.sales.all_data_types """).show(100,100)

In [ ]:
spark_polaris.sql("""select count(*) from polaris.sales.all_data_types """).show(100,100)

In [ ]:
spark_polaris.sql("""SHOW VIEWs  """).show(100,100)

## Create Iceberg Views

In [ ]:
spark_polaris.sql("""DROP VIEW if exists polaris.sales.all_data_types_view """)

In [ ]:
spark_polaris.sql("""CREATE OR REPLACE VIEW polaris.sales.all_data_types_view 
AS
SELECT COUNT(*) FROM polaris.sales.all_data_types""")


In [ ]:
spark_polaris.sql("""SELECT * FROM polaris.sales.all_data_types_view """).show(100,100)

In [ ]:
spark_polaris.sql("""insert into polaris.sales.all_data_types
SELECT * FROM polaris.sales.all_data_types""")

In [ ]:
spark_polaris.sql("""SELECT * FROM polaris.sales.all_data_types_view """).show(100,100)

# Time Travel on Table

In [ ]:
spark_polaris.sql(f"""
    SELECT committed_at, snapshot_id, parent_id 
    FROM polaris.sales.all_data_types.snapshots 
    ORDER BY committed_at DESC
""").show(100, 300)


In [ ]:
spark_polaris.sql(f""" SELECT count(*) FROM 
polaris.sales.all_data_types
FOR SYSTEM_TIME AS OF  '2025-02-09 09:19:13.659' """).show(100, 300)

# Time Travel on Views

In [ ]:
spark_polaris.sql(f"""
    SELECT committed_at, snapshot_id, parent_id 
    FROM polaris.sales.all_data_types.snapshots 
    ORDER BY committed_at DESC
""").show(100, 300)


In [ ]:
spark_polaris.sql(f""" SELECT * FROM polaris.sales.all_data_types_view
FOR SYSTEM_TIME AS OF  '2025-02-09 09:19:25.21' """).show(100, 300)
